In [1]:
import pandas as pd
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import uvicorn
import nest_asyncio


nest_asyncio.apply()


In [2]:
class HousePricePredictor:
    def __init__(self):
        self.model = LinearRegression()
        self.scaler = StandardScaler()
        self._train_model()
    
    def _train_model(self):
        df = pd.read_csv('train.csv')
        X = df[['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']]
        y = df['SalePrice']
        X_scaled = self.scaler.fit_transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
        self.model.fit(X_train, y_train)
    
    def predict(self, features):
        features_scaled = self.scaler.transform([features])
        prediction = self.model.predict(features_scaled)
        return prediction[0]


In [3]:
app = FastAPI()
predictor = HousePricePredictor()

class HouseFeatures(BaseModel):
    OverallQual: int
    GrLivArea: float
    GarageCars: int
    TotalBsmtSF: float
    FullBath: int
    YearBuilt: int

@app.post("/predict")
def predict_price(features: HouseFeatures):
    features_list = [
        features.OverallQual,
        features.GrLivArea,
        features.GarageCars,
        features.TotalBsmtSF,
        features.FullBath,
        features.YearBuilt,
    ]
    predicted_price = predictor.predict(features_list)
    return {"predicted_price": predicted_price}


In [ ]:
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8001)


INFO:     Started server process [28420]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)


INFO:     127.0.0.1:61826 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:61826 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:61853 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:61853 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:61854 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:61862 - "POST /predict HTTP/1.1" 200 OK


C:\python\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


INFO:     127.0.0.1:61873 - "GET / HTTP/1.1" 404 Not Found
